---
# THIS IS THE REAL EVAL

#### quick fix : 14, 71~81
#### quick fix 2 : find best tr cases and only include them in ts

In [ ]:
!pip3 install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
!pip3 install --quiet nibabel

---
# DICE

In [19]:
import torch
import nibabel as nib
import os
import numpy as np

In [99]:
def eval(inputs, targets, smooth=0.001):
    # Smooth prevents miss matches from being too significant but keeps the child of fraction above zero.
    # from torch.nn import functional as F
    # inputs = F.sigmoid(inputs) # uncomment if inputs is not between 0-1

    inputs = inputs.flatten()
    targets = targets.flatten()

    intersection = (inputs * targets).sum()
    dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)

    # return 1 - dice
    # return -torch.log(dice)
    return dice

def eval2(input, gt, smooth=0.00001):
    intersect = np.sum(np.logical_and(input, gt))
    union = np.sum(np.logical_or(input, gt))
    dice = (2. * intersect + smooth) / (union + intersect + smooth)
    return np.round(dice, 4)

def ensemble(pred_2d, pred_3d):
    from torch.nn import functional as F
    # INPUTS ARE PROBABILITIES MATRICES
    fn_class =  lambda x: 1.0 * (x > 0.5) # network output image를 binary class로 분류해주는 class function정의
    # compare tensors and return bigger num
    # compare = lambda x, y : 0 if x > y else 1
    
    # pred_2d = F.softmax(pred_2d)
    # pred_3d = F.softmax(pred_3d)

    # twoD = compare(pred_2d[0].transpose(1,2,0), pred_2d[1].transpose(1,2,0))
    # threeD = compare(pred_3d[0].transpose(1,2,0), pred_3d[1].transpose(1,2,0))

    pred_2d_flatten = pred_2d.flatten()
    pred_3d_flatten = pred_3d.flatten()

    new_pred = (pred_2d_flatten * pred_3d_flatten)/2
    new_label = fn_class(new_pred)

    return new_label

def init_imgs(path): # PACK UP PATHS
    pred2d_list = os.listdir(os.path.join(path, "pred2d"))
    pred3d_list = os.listdir(os.path.join(path, "pred3d"))
    gt_list = os.listdir(os.path.join(path, "b4_raw"))
    
    temp_list = []
    for thing in pred2d_list:
        if ".nii.gz" in thing:
            mpath = os.path.join(path, "pred2d")
            temp_list.append(os.path.join(mpath, thing))
    pred2d_list = temp_list
    
    temp_list = []
    for thing in pred3d_list:
        if ".nii.gz" in thing:
            mpath = os.path.join(path, "pred3d")
            temp_list.append(os.path.join(mpath, thing))
    pred3d_list = temp_list

    pnum_list = [os.path.basename(file).split(".nii.gz")[0] for file in pred3d_list]

    temp_list = []
    for p in gt_list:
        if p in pnum_list:
            seg = os.path.join(path, f"b4_raw/{p}/b4_reg_seg.nii.gz")
            temp_list.append(seg)
    gt_list = temp_list

    temp_list = [gt_list, pred2d_list, pred3d_list]

    return temp_list

In [100]:
path = "/workspace/dwseon/dat/"
l = init_imgs(path)
gt_list = l[0]
pred2d_list = l[1]
pred3d_list = l[2]

In [96]:
print (f"Len check : {len(gt_list), len(pred2d_list), len(pred3d_list)}")

Len check : (19, 0, 19)


---
### "Data supplement"

In [92]:
# for 'data supplement'
indiv_dice = {}
for i, gt in enumerate(gt_list):
    pNum = os.path.basename(gt.split("/b4_reg_seg.nii.gz")[0])
    gt_img = nib.load(gt).get_fdata().astype(int)
    #gt_img = torch.from_numpy(gt_img)
    for threeD in pred3d_list:
        pNum3D = os.path.basename(threeD).split(".nii.gz")[0]
        if pNum == pNum3D:
            img_3d = nib.load(threeD).get_fdata().astype(int)
            #img_3d = torch.from_numpy(img_3d)
            indiv_dice[pNum3D] = eval(img_3d, gt_img, smooth=0.00001)


In [93]:
test = ["082", "083", "084", "085", "086", "087", "088", "089", "090", "091", "092", "093", "094", "095", "096", "097", "098", "099", "100"]
d = []
zero = 0
one = 0
above_hf = 0
for key, val in indiv_dice.items():
    if val > 0 and val < 1:
        d.append((key, val))
        if key in test:
            print(f"Testcase {key}: {val}")
    if val == 0: 
        zero += 1
        if key in test:
            print(f"Testcase {key}: {val}")
    elif val == 1: 
        one += 1
        if key in test:
            print(f"Testcase {key}: {val}")
    if val > 0.5: above_hf +=1
print(f"score zero count at {zero}\nscore one count at {one}\nabove median {above_hf}")

Testcase 082: 4.9999750001249995e-06
Testcase 083: 1.0
Testcase 084: 1.0
Testcase 085: 0.7888198790555919
Testcase 086: 3.412969166792862e-08
Testcase 087: 0.4678899326655994
Testcase 088: 1.0
Testcase 089: 1.0
Testcase 090: 1.0
Testcase 091: 1.0
Testcase 092: 1.0
Testcase 093: 1.0
Testcase 094: 1.0
Testcase 095: 1.0
Testcase 096: 1.0
Testcase 097: 1.0
Testcase 098: 1.0
Testcase 099: 1.0
Testcase 100: 0.40650411329234853
score zero count at 0
score one count at 70
above median 86


In [94]:
# print(f"len: {len(d)}\ndice max: {np.max(d)}\ndice min: {np.min(d)}\ndice unique: {set(d)}")
for i in range(len(d)):
    for j in range(len(d)):
        if d[j][1] > d[i][1]:
            temp = d[i]
            d[i] = d[j]
            d[j] = temp
d

[('086', 3.412969166792862e-08),
 ('066', 2.777777006173054e-07),
 ('005', 4.1666649305562796e-07),
 ('051', 3.3333222222592593e-06),
 ('004', 3.3333222222592593e-06),
 ('049', 3.3333222222592593e-06),
 ('082', 4.9999750001249995e-06),
 ('070', 9.99990000099999e-06),
 ('081', 9.99990000099999e-06),
 ('026', 0.19741101622312274),
 ('077', 0.24338628341871518),
 ('100', 0.40650411329234853),
 ('023', 0.46153887573932634),
 ('087', 0.4678899326655994),
 ('030', 0.5080583307132899),
 ('075', 0.5547918563677968),
 ('014', 0.6589469875843944),
 ('032', 0.7021276701373167),
 ('013', 0.7363323743867413),
 ('085', 0.7888198790555919),
 ('072', 0.8192771193206555),
 ('035', 0.829931975681429),
 ('068', 0.8299319766455334),
 ('058', 0.8481375401679788),
 ('007', 0.8607595112962644),
 ('057', 0.8744588771762147),
 ('019', 0.8774193627471379),
 ('010', 0.9375000097656235),
 ('011', 0.9444444598765389),
 ('078', 0.948115784008106)]

"Supplement" ends
<hr>

In [101]:
threeD_dice = {}
twoD_dice = {}
ensemble_dice = {}
prob2d, prob3d = 0, 0
ensem_out = os.path.join(path, "ensemble")
if not os.path.isdir(ensem_out):
    os.makedirs(ensem_out)
for i, gt in enumerate(gt_list):
    gt_np = nib.load(gt).get_fdata().astype(int)
    gt_img = torch.from_numpy(gt_np)
    for twoD in pred2d_list:
        if os.path.basename(gt.split("/b4_reg_seg.nii.gz")[0]) == os.path.basename(twoD).split(".nii.gz")[0]:
            # LOAD IMAGE
            pred2d_img = nib.load(twoD).get_fdata().astype(int)
            ev2 = eval2(pred2d_img, gt_np)
            pred2d_img = torch.from_numpy(pred2d_img)
            base = os.path.basename(twoD).split(".nii.gz")[0]
            # DICE
            twoD_dice[base] = (eval(pred2d_img, gt_img, smooth=0.00001), ev2)
            # SOFTMAX
            # prob2d = np.load(os.path.join(path, f"pred2d/{base}.npz"), allow_pickle=True)["softmax"] # this has 2 channels tho
    for threeD in pred3d_list:
        if os.path.basename(gt.split("/b4_reg_seg.nii.gz")[0]) == os.path.basename(threeD).split(".nii.gz")[0]:
            # LOAD IMAGE
            
            pred3d_img = nib.load(threeD).get_fdata().astype(int)
            ev2 = eval2(pred3d_img, gt_np)
            pred3d_img = torch.from_numpy(pred3d_img)
            base = os.path.basename(threeD).split(".nii.gz")[0]
            # DICE
            threeD_dice[base] = (eval(pred3d_img, gt_img, smooth=0.00001), ev2)
            # SORFTMAX
            # prob3d = np.load(os.path.join(path, f"pred3d/{base}.npz"), allow_pickle=True)["softmax"] # so let's use them separately
    # ENSEMBLE
    # ens_res = ensemble(prob2d, prob3d) # assuming [0] is pred and [1] is gt
    # ens_nii = nib.Nifti1Image(ens_res.reshape(prob2d[1].transpose(1,2,0).shape), affine=np.eye(4))
    # nib.save(ens_nii, os.path.join(ensem_out, f"{base}.nii.gz"))
    # ensemble_dice[base] = eval(torch.from_numpy(ens_res.astype(int)), gt_img, smooth=0.00001)

In [105]:
def avg_dice(dic):
    tot = 0
    for thing in dic:
        tot += dic[thing][0]
    if len(dic) != 0:
        tot = tot / len(dic)
    else:
        tot = "This plan contains 0 data."
    return tot

In [106]:
print(f"Average of \n- 2d dice: {avg_dice(twoD_dice)}\n- 3d dice: {avg_dice(threeD_dice)}\n- Ensemble: {avg_dice(ensemble_dice)}")

Average of 
- 2d dice: This plan contains 0 data.
- 3d dice: 0.9459847807884216
- Ensemble: This plan contains 0 data.


---
# Rate and Recall

In [107]:
from sklearn.metrics import confusion_matrix as kungfu
import matplotlib.pyplot as plt

In [111]:
print(pred3d_list[0])
print(gt_list[0])

/workspace/dwseon/dat/pred3d/007.nii.gz
/workspace/dwseon/dat/b4_raw/032/b4_reg_seg.nii.gz


In [110]:
n_inspect = 1
for i in range(len(pred3d_list)):
    pred = nib.load(pred3d_list[i])
    gt = nib.load(gt_list[i])

    if i >= n_inspect:
        break

'/workspace/dwseon/dat/pred3d/007.nii.gz'

---
BELOW ARE TEST CELLS
---

In [160]:
img = nib.load(os.path.join(path, "pred2d/087.nii.gz")).get_fdata().astype(int)

In [102]:
a = torch.from_numpy(img)

In [103]:
a.unique()

tensor([0])

In [85]:
b = a.flatten()
b.shape

torch.Size([39845888])

In [192]:
aa = np.load("/workspace/dwseon/dat/pred2d/087.pkl", allow_pickle=True)

In [191]:
aa = aa["softmax"][0].transpose(1,2,0)
aa.shape

(1024, 1024, 38)

In [164]:
# shape don't matter but the len
data = aa['softmax']

In [165]:
data.shape

(2, 38, 1024, 1024)